In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv, datetime, os, random
import matplotlib.dates as mdates
import pandas as pd
from sklearn.metrics import r2_score
from tqdm import tqdm
from datetime import datetime as DT


def TimeMoveAvg(time, data, window, overlap):
    """
    """
    every, avgtime, avgdata = [], [], []
    t0 = time[0]
    while t0 < time[-1]:
        every.append([t0, t0 + datetime.timedelta(seconds=window)])
        t0 += datetime.timedelta(seconds=overlap)
    
    for j in range(len(every)): 
        d_ = []
        for ti,da in zip(time, data):
            if ti>=every[j][0] and ti<every[j][1]:
                d_.append(da)
        
        if len(d_)>0:
            avgdata.append(sum(d_)/len(d_))
            avgtime.append(every[j][0])
    
    return np.array(avgtime), np.array(avgdata)


def AvgOrSelect(time1, time2, d1, d2):
    """
    """
    # Get start and end time to iterate over
    timezero = time1[0]  if time1[0]  < time2[0]  else time2[0]
    timeneg1 = time1[-1] if time1[-1] > time2[-1] else time2[-1]

    # Create dictionaries for easy access
    DATA1 = { time1[i]:d1[i] for i in range(len(time1)) }
    DATA2 = { time2[i]:d2[i] for i in range(len(time2)) }

    # print(int((timeneg1-timezero).total_seconds() + 100))
    timesteps = [ timezero + datetime.timedelta(seconds=j) for j in range(int((timeneg1-timezero).total_seconds() + 100)) ]
    OUTPUT_TEMP = { timesteps[i]:None for i in range(len(timesteps)) }

    for n in range(len(timesteps)):

        # Case where we averaged the two
        if timesteps[n] in DATA1.keys():
            if timesteps[n] in DATA2.keys():
                OUTPUT_TEMP[timesteps[n]] = (DATA1[timesteps[n]] + DATA2[timesteps[n]])/2.0
            
            # Only in Data 1
            else:
                OUTPUT_TEMP[timesteps[n]] = DATA1[timesteps[n]]
        
        # Only in Data 2
        elif timesteps[n] in DATA2.keys():
            OUTPUT_TEMP[timesteps[n]] = DATA2[timesteps[n]]

    OUTPUT = dict()
    for k,v in OUTPUT_TEMP.items():
        if v:
            OUTPUT[k] = v
    ST_  = sorted(OUTPUT.items())

    return [j[0] for j in ST_], [j[1] for j in ST_]


def MakeDataFromText(filepath:str)->dict:
    """
    Helper function to parse text files in IFM proprietary format.
    """
    with open(filepath, 'r') as file:
        reader = csv.reader(file)

        db = dict()                          # Dictionary that will hold k,v pairs of 
        recent_EPC = ''                      # The most recent EPC that was read, and data is being assigned to
        Time_Or_Data = True                  # Time is true, Data is false
        reached_24 = False

        for l in reader:                     # Iterate over entire text file
            line = l[0]                      # Each line only has one token, so we select it
            
            if len(line)==24:                # If it is an EPC
                db[line] = [[],[]]           # Create new k,v pair in database
                recent_EPC = line            # Current EPC to add to is selected
            
            elif line == '[':                # If start bracket, beginning to collect an array of data
                data = []

            elif line == ']':                # End bracket means end of data array
                if len(data)!=0:
                    if Time_Or_Data:
                        db[recent_EPC][0] = data
                        Time_Or_Data = False
                    else:
                        db[recent_EPC][1] = data
                        Time_Or_Data = True
                data = []

            elif line.count(":")==2:
                h,m,s = line.split(':')      # Parse timestamp into hours, minutes, seconds

                if int(h)==23:
                    reached_24 = True        # If end of day reached, must move to Day 2

                if reached_24 and int(h) < 12:
                    day=2
                else:
                    day=1

                data.append(datetime.datetime(1970, 2, day, int(h), int(m), int(s)))
            
            elif line ==' ':               # If blank space, then we skip to parse next line
                pass

            else:                          # If it is a data point
                try:
                    data.append(float(line))
                except:
                    pass                   # Error catching case just in case, so annoying errors don't crash program
    return db


def MakeDict(data:list):
    """
    """
    t1234 = dict()
    for d in data:
        for k,v in d.items():
            if k in t1234: # if tag is already in dict
                for i in range(len(v[0])):
                    if v[0][i] in t1234[k]:
                        t1234[k][v[0][i]].append(v[1][i])
                    else:
                        t1234[k][v[0][i]] = [v[1][i]]
            else:             # if tag is not in dict
                t1234[k] = dict()
                for i in range(len(v[0])):
                    t1234[k][v[0][i]] = [v[1][i]]
    for k,v in t1234.items():
        for k2,v2 in v.items():
            t1234[k][k2] = sum(v2)/len(v2)
    return t1234


def Chop(time, data, delin1, delin2):
    """
    Chop a set of lists based on the time frame given.
    """
    new_time = [ time[j] for j in range(len(time)) if time[j] > delin1 and time[j] < delin2 ]
    new_data = [ data[j] for j in range(len(time)) if time[j] > delin1 and time[j] < delin2 ]
    return new_time, new_data

In [ ]:
# r1 = MakeDataFromText('No Chamber/RSSI_11785.csv')
# t1 = MakeDataFromText('No Chamber/tags_11785.csv')
# r2 = MakeDataFromText('With Chamber/RSSI_14317.csv')
# t2 = MakeDataFromText('With Chamber/tags_14317.csv')
# r3 = MakeDataFromText('With Chamber/RSSI_65779.csv')
# t3 = MakeDataFromText('With Chamber/tags_65779.csv')
# r4 = MakeDataFromText('With Chamber/RSSI_91313.csv')
# t4 = MakeDataFromText('With Chamber/tags_91313.csv')

# RSSI_NC = MakeDict( [r1] )
# TEMP_NC = MakeDict( [t1] )
# RSSI_WC = MakeDict( [r2,r3,r4] )
# TEMP_WC = MakeDict( [t2,t3,t4] )